In [2]:
import serial
import time
import numpy as np
import tdwf
import matplotlib.pyplot as plt
import scipy.optimize as so
import psutil, os, sys


# -[Pre-Processing work]-------------------------------------------
p = psutil.Process(os.getpid())
os_used = sys.platform
if os_used == "win32":  # Windows (either 32-bit or 64-bit)
    p.nice(psutil.REALTIME_PRIORITY_CLASS)
elif os_used == "linux":  # linux
    p.nice(psutil.IOPRIO_HIGH)
else:  # MAC OS X or other
    p.nice(-20)  



# -[Parameters]-------------------------------------------

# 1. hardware parameters
port = 'COM7'  # serial port for the PR59
R = 9.971e3 # shunt resistance [Ohm]
sigR = 299 # shunt resistance tolerance [Ohm]
expected_Vj = 0.7 # expected junction Voltage [V]

# 2. Measurement parameters
max_stabilizing_time = 360 # max stabilizing time
nT = 2 # number of temperature points to measure
T_range = [10, 11] # temperature range [°C]
temperatures = np.linspace(T_range[0], T_range[1], nT) # temperatures at which to measure
flag_return_T = True # if true the measurement are done both increasing and lowering the temperature
admitted_deviation = 0.05 # [% of desired temp] maximum deviation from the set temperature to be considered stable
stabilized_samples = 200 # number of temperature samples to be stabilized before measuring the IV curve

n_pulls = 3 # number of pulls to be done for each temperature
measuring_delay = 10 # [s] delay between the pulls
nv = 50 # number of voltage points to measure
V_range = [-5, 5] # voltage range [V]
offsets = np.linspace(V_range[0], V_range[1], nv) # voltages at which to measure
flag_return_V = True # if true the IV charatestics is measured in both directions

# 3. Plot Parameters
flag_show = True # Plotting the data while measuring
flag_save = True # Saving the data in a file, not sure why you should ever turn it off but ehy, you can

# 4. Data Saving Parameters
Directory = f"IV-T_dependence_{time.strftime('%Y%m%d_%H%M%S')}"
parent_dir = "../../../Data"
path = os.path.join(parent_dir, Directory)
os.mkdir(path)
print(f"Directory {path} created")



# -[Serial Connection initialization and startup]-------------------------------------------

# 1. Connection setup
ser = serial.Serial()
ser.port = port
ser.baudrate = 115200   
ser.timeout = 1
ser.rts = False
ser.dtr = False
ser.open()

# 2. Communcation function definition
# sending and receiving data to the device
def query(message, flagR=True, flagW=True):
    if flagW:    
        message += '\n\r'
        ser.write(message.encode('utf-8'))
        res = ser.readline().decode()
    if flagR:
        res = ser.readline().decode()
        return res
    
# 3. Testing the connection
swap = query("$V?")
print(swap)
time.sleep(1)


# -[PID Configuration]-------------------------------------------
# technically should be already done in device, leave here just in case


# -[AD2 Connection initialization and startup]-------------------------------------------

# 1. Connection
ad2 = tdwf.AD2()

# 2. Function generator setup
wavegen = tdwf.WaveGen(ad2.hdwf)
wavegen.w1.func = tdwf.funcDC
wavegen.w1.start()

# 3. Analog input setup
scope = tdwf.Scope(ad2.hdwf)
scope.fs = 1e6
scope.npt = 8192
scope.ch1.rng = 10
scope.ch2.rng = 10


# -[Plot initialization]-------------------------------------------
#fig, [[ax1, ax3, ax5], [ax2, ax4a, ax6]] = plt.subplots(2, 3, figsize=(10, 8), dpi=100)
#ax4b = ax4a.twinx()
#fig.canvas.manager.set_window_title('IV Measurement - T dependence')
#fig.suptitle('IV Measurement - T dependence', fontsize=18)
print(offsets)
print(temperatures)




Directory ../../../Data\IV-T_dependence_20250403_101055 created
SC_v2.9

Dispositivo #1 [SN:210321ABE62D, hdwf=1] connesso!
Configurazione #1
[-5.         -4.79591837 -4.59183673 -4.3877551  -4.18367347 -3.97959184
 -3.7755102  -3.57142857 -3.36734694 -3.16326531 -2.95918367 -2.75510204
 -2.55102041 -2.34693878 -2.14285714 -1.93877551 -1.73469388 -1.53061224
 -1.32653061 -1.12244898 -0.91836735 -0.71428571 -0.51020408 -0.30612245
 -0.10204082  0.10204082  0.30612245  0.51020408  0.71428571  0.91836735
  1.12244898  1.32653061  1.53061224  1.73469388  1.93877551  2.14285714
  2.34693878  2.55102041  2.75510204  2.95918367  3.16326531  3.36734694
  3.57142857  3.7755102   3.97959184  4.18367347  4.3877551   4.59183673
  4.79591837  5.        ]
[10. 11.]


In [3]:
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt 

flag_testing = False
temp = []

# -[Plot initialization]-------------------------------------------
fig, [[ax1, ax3, ax5], [ax2, ax4a, ax6]] = plt.subplots(2, 3, figsize=(10, 8), dpi=100)
ax4b = ax4a.twinx()
fig.canvas.manager.set_window_title('IV Measurement - T dependence')
fig.suptitle('IV Measurement - T dependence', fontsize=18)


# -[Temperature Measurement Function]-------------------------------------------
def measure_temp():
    if flag_testing:
        data = np.sin(len(temp)/10) * .10 + np.sin(len(temp)/2) * .05 + temp_desired# test data with obvious pattern
    else:
        data = float(query("$R100?"))
    
    temp.append(data)
    return data


# -[IV Measurement Function]-------------------------------------------
def measureIV(list_offsets):
    offsets = list_offsets.copy()
    flag_first = True
    Vd = np.full((nv, 2), np.nan) # diode voltage data
    ErrVd = np.full((nv, 2), np.nan) # diode voltage error
    Vr = np.full((nv, 2), np.nan) # resistor voltage data
    ErrVr = np.full((nv, 2), np.nan) # resistor voltage error
    Id = np.full((nv, 2), np.nan) # current data
    ErrId = np.full((nv, 2), np.nan) # current error
    Td = np.full((nv, 2), np.nan) # temperature data

    for ar_V in range(2 if flag_return_V else 1):
        for ii in range(nv):
            if ar_V == 0:
                i = ii
            else:
                i = nv - ii - 1

            # 1. Set voltage
            wavegen.w1.offs = offsets[i]
            wavegen.w1.start()
            time.sleep(.2) # wait for the voltage to stabilize

            # 2. Measure temperature
            measure_temp()
            update_Tplots()

            # 2. Voltage sampling and automatic fitting
            scope.sample()

            fitfuncDC = lambda x, o: np.array( [o for i in x] )
            pp1, cov1 = so.curve_fit(fitfuncDC, scope.time.vals, scope.ch1.vals, p0=[wavegen.w1.offs])
            pp2, cov2 = so.curve_fit(fitfuncDC, scope.time.vals, scope.ch2.vals, p0=[wavegen.w1.offs])

            Td[i, ar_V] = temp[-1] # temperature
            Vd[i, ar_V] = pp1[0] # diode voltage
            ErrVd[i, ar_V] = np.sqrt(cov1[0][0]) # diode voltage error
            Vr[i, ar_V] = pp2[0] # resistor voltage
            ErrVr[i, ar_V] = np.sqrt(cov2[0][0]) # resistor voltage error
            Id[i, ar_V] = pp2[0] / R # current
            ErrId[i, ar_V] = np.sqrt( cov2[0][0]/R**2 + sigR**2 * pp2[0]**2 / R**4 ) # current error
            print(pp2[0])

            if flag_show:
                if flag_first:
                    flag_first = False
                    hp1_true, = ax1.plot(1000*scope.time.vals, scope.ch1.vals, "-", label="Ch1", color="tab:orange")
                    hp1_sym, = ax1.plot(1000*scope.time.vals, fitfuncDC(1000*scope.time.vals, *pp1), "--", label="Fit Ch1", color="#00e7e7")
                    ax1.grid(linestyle='-.')
                    ax1.set_ylabel("Vd [V]", fontsize=15)
                    ax1.set_ylim([-1.2*max(offsets), 1.2*max(offsets)])
                    ax1.set_title(f"Time plots", fontsize=15)


                    hp2_true, = ax2.plot(1000*scope.time.vals, scope.ch2.vals, "-", label="Ch2", color="tab:blue")
                    hp2_sym, = ax2.plot(1000*scope.time.vals, fitfuncDC(1000*scope.time.vals, *pp2), "--", label="Fit Ch2", color="#e500a7")
                    ax2.grid(linestyle='-.')
                    ax2.set_xlabel("Time [msec]", fontsize=15)
                    ax2.set_ylabel("Vr [A]", fontsize=15)
                    ax2.set_ylim([-1.2*max(offsets), 1.2*max(offsets)])


                    hp3A, = ax3.plot(Vd[:, 0], Id[:, 0], ".", markerfacecolor = "none", label="Go Run", color="tab:purple")
                    if flag_return_V:
                        hp3R, = ax3.plot(Vd[:, 1], Id[:, 1], "v",  markerfacecolor = "none", label="Return Run", color="tab:cyan")
                    
                    ax3.grid(linestyle='-.')
                    ax3.set_xlabel("Vd [V]", fontsize=15)
                    ax3.set_ylabel("Id = Vr/R [A]", fontsize=15)
                    ax3.set_xlim([min(offsets)-0.5, expected_Vj])
                    ax3.set_ylim([min(offsets)/R-0.2/R, max(offsets)/R])
                    ax3.legend(loc='upper left')
                    ax3.set_title(f"I-V Charateristic", fontsize=15)


                    hp4VA, = ax4a.plot(offsets, Vd[:, 0], ".", markerfacecolor = "none", label="Vd Go", color="tab:orange")
                    if flag_return_V:
                        hp4VR, = ax4a.plot(offsets, Vd[:, 1], "v",  markerfacecolor = "none", label="Vd Return", color="tab:orange")                
                    ax4a.grid(linestyle='-.')
                    ax4a.set_xlabel("Vcc [V]", fontsize=15)
                    ax4a.set_ylabel("Vd [V]", fontsize=15)
                    ax4a.tick_params(axis='y', labelcolor="tab:orange")
                    ax4a.set_xlim([min(offsets)-0.5, 1.2*max(offsets)])
                    ax4a.set_ylim([min(offsets)-0.5, 1.2*expected_Vj])

                    
                    hp4IA, = ax4b.plot(offsets, Id[:, 0], ".", markerfacecolor = "none", label="Id Go", color="tab:blue")
                    if flag_return_V:
                        hp4IR, = ax4b.plot(offsets, Id[:, 1], "v",  markerfacecolor = "none", label="Id Return", color="tab:blue")
                    ax4b.set_ylabel("Id = Vr/R [A]", fontsize=15)
                    ax4b.tick_params(axis='y', labelcolor="tab:blue")
                    ax4b.set_ylim([min(offsets)/R-0.2/R, max(offsets)/R])



#                    plt.tight_layout()
#                    plt.show(block=False)    
                else:
                    hp1_true.set_xdata(1000*scope.time.vals)
                    hp1_true.set_ydata(scope.ch1.vals)
                    hp1_sym.set_xdata(1000*scope.time.vals)
                    hp1_sym.set_ydata(fitfuncDC(scope.time.vals, *pp1))
                    hp2_true.set_xdata(1000*scope.time.vals)
                    hp2_true.set_ydata(scope.ch2.vals)
                    hp2_sym.set_xdata(1000*scope.time.vals)
                    hp2_sym.set_ydata(fitfuncDC(scope.time.vals, *pp2))
                    if ar_V==0:
                        hp3A.set_xdata(Vd[:, 0])
                        hp3A.set_ydata(Id[:, 0])

                        hp4VA.set_xdata(offsets[:])
                        hp4VA.set_ydata(Vd[:, 0])
                        hp4IA.set_xdata(offsets[:])
                        hp4IA.set_ydata(Id[:, 0])
                    else:
                        hp3R.set_xdata(Vd[:, 1])
                        hp3R.set_ydata(Id[:, 1])

                        hp4VR.set_xdata(offsets[:])
                        hp4VR.set_ydata(Vd[:, 1])
                        hp4IR.set_xdata(offsets[:])
                        hp4IR.set_ydata(Id[:, 1])

    # data saving
    if flag_save:
        info = f"IV Measurement - T = {temp_desired:.2f} °C - Pull number {pull}\n"
        if ar_V == 0:
            data_to_save = np.c_[Td[:, 0], offsets, Vd[:, 0], ErrVd[:, 0], Vr[:, 0], ErrVr[:, 0], Id[:, 0], ErrId[:, 0]]
            info += f"Td_go[°C]\tVcc\tVd_go[V]\tErrVd_go[V]\tVr_go[V]\tErrVr_go[V]\tId_go[A]\tErrId_go[A]\n"
        else:
            data_to_save = np.c_[Td, offsets, Vd, ErrVd, Vr, ErrVr, Id, ErrId]
            info += f"Td_go[°C]\tTd_return[°C]\tVcc\tVd_go[V]\tVd_return[V]\tErrVd_go[V]\tErrVd_return[V]\tVr_go[V]\tVr_return[V]\tErrVr_go[V]\tErrVr_return[V]\tId_go[A]\tId_return[A]\tErrId_go[A]\tErrId_return[A]\n"

        filename = f"IV_T{temp_desired:.2f}_V{offsets[0]:.2f}_{offsets[-1]:.2f}_{pull}Pull"


        np.savetxt(path+'/'+filename+".txt", data_to_save, delimiter="\t", header=info, fmt="%s")


    return Td, Vd, ErrVd, Vr, ErrVr, Id, ErrId


# -[Temperature Plots Update]---------------------------------------------
def update_Tplots():
    ax5.clear()
    ax6.clear()

    fig.suptitle("User interaction: x|c|space|escape|e(DONT BE SILLY, GOOSE)")

    ax5.set_title("Full data acquisition")
    ax5.plot(np.arange(0, len(temp)), temp, "-", label="Temp Data", color="tab:blue")
    ax5.set_xlabel("Sample", fontsize=15)
    ax5.set_ylabel("Temperature [°C]", fontsize=15)
    ax5.legend(loc='upper right')
    ax5.grid(linestyle='-.')

    secondGraphMax = min(200, len(temp))
#    if(len(temp) < 200):
#        secondGraphMax = len(temp)

    ax6.set_title("Latest 200 samples")
    ax6.plot(np.arange(len(temp)-secondGraphMax, len(temp)), temp[-secondGraphMax:], "-", label="Latest Temp Data", color="tab:orange")
    ax6.set_xlabel("Sample", fontsize=15)
    ax6.set_ylabel("Temperature [°C]", fontsize=15)
    ax6.annotate(f"Latest Value [°C]: {temp[-1]:.2f}", (0.4,0.96), xycoords='axes fraction', fontsize=12)
    ax6.legend(loc='upper right')
    ax6.grid(linestyle='-.')

    fig.canvas.draw()
    fig.canvas.flush_events()
    plt.show(block=False)






# -[Main Loop]-------------------------------------------
for ar_T in range(2 if flag_return_T else 1):
    if ar_T == 1:
        temperatures = np.flip(temperatures)

    for temp_desired in temperatures:
        temp_skip = False
        # 1. Set temperature
        if not flag_testing:
            print(int(temp_desired))
            query(f"$R0={int(temp_desired)}")

        starting_waiting_time = time.time()
        while True:
            # 2. Wait for temperature to stabilize
            measure_temp()


            if(len(temp) >= stabilized_samples):
                if abs(max(temp[-stabilized_samples:]) - temp_desired) < admitted_deviation*temp_desired and abs(min(temp[-stabilized_samples:]) - temp_desired) < admitted_deviation*temp_desired:
                    break
            
            update_Tplots()

            if time.time() - starting_waiting_time > max_stabilizing_time:
                # avoid measures and go to next temperature
                temp_skip = True

        if(temp_skip):
            break
        
        pull = 0
        starting_measuring_time = time.time()
        while pull < n_pulls:
            if time.time() - starting_measuring_time > measuring_delay:
                ax1.clear()
                ax2.clear()
                ax3.clear()
                ax4a.clear()
                ax4b.clear()
                fig.suptitle(f"IV Measurement - T = {temp_desired:.2f} °C", fontsize=18)
                measureIV(offsets)
                pull += 1
                starting_measuring_time = time.time()
            else:
                measure_temp()
                update_Tplots()
    









10
-0.017932219819101647
-0.01751648218972312
-0.017355918305796025
-0.017451892750851596
-0.016343410660457792
-0.018187848445636447
-0.019352732579372343
-0.018931081821085324
-0.017783937039639767
-0.01708164063101483
-0.018594034126333456
-0.019761647391408847
-0.017560603140360453
-0.01771707332716063
-0.018613138043481862
-0.01870911248986191
-0.01885785013436818
-0.01905753156150962
-0.017364560555345347
-0.017761194280293078
-0.019689780272889937
-0.019120301577229385
-0.017541954081921073
-0.01944142933991024
-0.019315889309230828
-0.019645204464570486
-0.015420510467909265
0.0779785448861594
0.2494953401591688
0.434286171583991
0.6248359264529194
0.8193283650059552
1.0147350631241574
1.2117865195945603
1.4098759513050971
1.6106021807331674
1.8102408514092807
2.0112891183096995
2.210225492577188
2.412919423634136
2.6134923687740055
2.813865175721829
3.0156289298312937
3.217645128569507
3.4197550274762283
3.61838391963735
3.8224560630356472
4.02200785715591
4.2239048838351465
4

In [ ]:
ser.close()

In [1]:
ad2.close()

NameError: name 'ad2' is not defined